# QDAC II QICK Trig Test

## Import Libraries

In [4]:
import time
import json
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Union, Any
from pprint import pprint
from time import sleep

from qcodes import Parameter, Measurement, Station, load_or_create_experiment, initialise_or_create_database_at
from qcodes.instrument.channel import ChannelList

from pyvisa.constants import StopBits, Parity

from qstl_instruments.qstl_qdac2 import QSTL_QDac2

from qick import *
from qick.averager_program import QickSweep
from qick.pyro import make_proxy

# Qick version : 0.2.357
(soc, soccfg) = make_proxy("192.168.2.99")

rm = pyvisa.ResourceManager()
rm.list_resources()

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8888
myqick PYRO:obj_3cde8b18d45b4d708deec1c942f4b4c0@192.168.2.99:34289


('USB0::0x0957::0x1780::MY60101437::INSTR',
 'ASRL1::INSTR',
 'ASRL3::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL20::INSTR',
 'ASRL21::INSTR',
 'ASRL22::INSTR',
 'ASRL23::INSTR',
 'ASRL24::INSTR',
 'ASRL25::INSTR',
 'ASRL26::INSTR',
 'ASRL27::INSTR')

## QICK Program

In [11]:
class TrigProgram(AveragerProgram):
    def initialize(self):
        cfg = self.cfg
        # Declare DC generation channel
        self.declare_gen(
            ch      = 0,        # Channel
            nqz     = 1,        # Nyquist Zone
        )
        self.declare_readout(
            ch      = 0,
            length  = 160,
        )
        self.set_pulse_registers(
            ch      = 0,        # Generator channel
            style   = "const",  # Output is constant
            gain    = 0,
            mode    = "periodic",
            length  = 32,
            freq    = 0,
            phase   = 0,
            phrst   = 0,
        )
        self.set_readout_registers(
            ch      = 0,        # Readout channel
            freq    = 0,        # Readout DDS frequency
            length  = 16,       # Readout DDS multiplication length
            phrst   = 0,        # Readout DDS phase reset
        )
        self.synci(100)
    def body(self):
        cfg = self.cfg
        self.pulse(
            ch      = 0,        # Generator channel
            t       = 100       # Pulse will be output @ sync_t + 100
        )
        self.readout(
            ch      = 0,        # Readout channel
            t       = 100       # Readout DDS will start multiplication
                                # @ sync_t + 100
        )
        self.trigger(
            adcs    = [0],      # Readout channels
            pins    = [0],
            adc_trig_offset = 50, # Readout will capture the data @ sync_t + 50
            width   = 100
        )
        self.sync_all()


## Trigger Execution

In [13]:
cfg = {
    # Experiment Setup
    "reps" : 1,
    "duration" : 1500,
    "expts" : 1,
}
prog = TrigProgram(
    soccfg,
    cfg
)
prog.acquire(soc, progress = False)

([array([0.])], [array([0.])])